# PostgreSQL as Vector Database: Getting Started With pgvector

Pgvector is a PostgreSQL extension that provides the database with essential capabilities needed for a vector database. With pgvector, you can efficiently store vectors/embeddings in PostgreSQL, perform vector similarity searches, optimize data access with IVFFlat and HNSW indexes, and much more.

## Prerequisites

* [Docker](https://www.docker.com)
* Python and pip.
* [OpenAI API key](https://platform.openai.com).

## Install Required Modules

The notebook uses the following libraries:

* `openai` - provides access to the OpenAI Embedding API.
* `psycopg2` - PostgreSQL database driver for Python.
* `wget` - allows for downloading files and datasets.

Install the libraries with pip:

In [ ]:
! pip install openai psycopg2 wget

## Start PostgreSQL With pgvector

The fastest way to begin using PostgreSQL as a vector database is by creating a database container with the pre-installed pgvector extension. Run the command below to start PostgreSQL with pgvector in Docker:

In [ ]:
! docker compose up -d

Enable the pgvector extension by connecting to the database instance from within the container with `psql` and running the `CREATE EXTENSION vector` command:

In [ ]:
# Wait for Postgres to be ready
! while ! docker exec -it postgres-pgvector pg_isready -U postgres; do sleep 1; done

# Enable the pgvector extension
! docker exec -it postgres-pgvector psql -U postgres -c 'CREATE EXTENSION vector'

## Provide OpenAI API Key

Provide your OpenAI API key by setting it as the `OPENAI_API_KEY` environment variable and run the code snippet below. If the variable is not set, you'll be prompted to enter the key:

In [ ]:
import os
import openai
from getpass import getpass

openai_key = os.getenv('OPENAI_API_KEY')

if (openai_key == None):
    openai_key = getpass('Provide your OpenAI API key: ')

if (not openai_key):
    raise Exception('No OpenAI API key provided. Please set the OPENAI_API_KEY environment variable or provide it when prompted.')

openai.api_key = openai_key

print('OpenAI API key set.')

## Load Sample Dataset

The notebook uses a [movies dataset](https://huggingface.co/datasets/denismagda/movies/blob/main/README.md) comprising over 45,000 movies and 26 million ratings from more than 270,000 users. The dataset includes pre-generated embeddings for the movies' overviews, which were generated with the OpenAI `text-embedding-ada-002` model.

First, download the dataset:

In [ ]:
import wget

schema_file = "https://huggingface.co/datasets/denismagda/movies/raw/main/movie_schema.sql"
data_file = "https://huggingface.co/datasets/denismagda/movies/resolve/main/movie_data_with_openai_embeddings.sql"

print('Downloading the schema file...')
wget.download(schema_file)

# This file is 900MB, so it might take a minute to download it
print('Downloading the data file...')
wget.download(data_file)

print('Finished downloading the files.')

Second, open a database connection using the psycopg2 driver:

In [ ]:
import psycopg2

print('Connecting to PostgreSQL...')
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=password")
    
cursor = conn.cursor()

print('Successfully connected to PostgreSQL.')

Finally, load the dataset into Postgres:

In [ ]:
print('Creating the schema...')
schema_file = open('movie_schema.sql', 'r')
cursor.execute(schema_file.read())
conn.commit()

print('Loading the data. It might take a minute...')
data_file = open('movie_data_with_openai_embeddings.sql', 'r')
cursor.execute(data_file.read())
conn.commit()

cursor.execute('SELECT COUNT(*) FROM movie')
result = cursor.fetchone()

print(f'{result[0]} movies loaded.')

## Perform Vector Similarity Search

The dataset already stores a vectorized representation of movies' overviews in the `overview_vector` column. Each vector is a 1536-dimensional embedding generated with the OpenAI `text-embedding-ada-002` model.

First, define several functions that generate vectors for user prompts and configure the matching threshold and count parameters for the similarity search:

In [10]:
# Converting the prompt to the pgvector embedding
def get_embedding(prompt):
    response = openai.embeddings.create(
        input=prompt,
        model='text-embedding-ada-002')

    embedding = response.data[0].embedding

    # Converting the embedding to the pgvector and returning it
    return '[' + ','.join(map(str, embedding)) + ']'

# Getting the matching threshold for the similarity search
def get_matching_threshold():
    return 0.7

# Getting the number of matching movies to return
def get_matching_count():
    return 3

Second, find the most relevant movies for a provided user prompt by calculating the cosine distance (`<=>`) between the prompt's and movies' embeddings:

In [ ]:
user_prompt = 'A movie about a space adventure.'

prompt_vector = get_embedding(user_prompt)

cursor.execute(
    'SELECT title, overview '
    'FROM movie WHERE 1 - (overview_vector <=> %(prompt_vector)s) >= %(match_threshold)s '
    'ORDER BY overview_vector <=> %(prompt_vector)s LIMIT %(match_cnt)s',
    {'prompt_vector': prompt_vector, 'match_threshold': get_matching_threshold(), 'match_cnt': get_matching_count()}
    )

result = cursor.fetchall()

for row in result:
    print(row)

## Pre-Filter Data Before Similarity Search

As a general-purpose relational database, PostgreSQL allows you to pre-filter data before a vector search is started. You can pre-filter by specifying a condition on non-vector columns in the `WHERE` clause of a query statement.

For instance, imagine the user selecting the `Science Fiction` category and asking to suggest movies with a rating of `7` or higher. Then, the user prompts for `A movie about a space adventure`. The final SQL query can look as follows:

In [ ]:
user_prompt = 'A movie about a space adventure.'

prompt_vector = get_embedding(user_prompt)

cursor.execute(
    'SELECT title, vote_average, genres '
    'FROM movie WHERE vote_average >= 7 '
    'AND genres @> \'[{"name": "Science Fiction"}]\' '
    'AND 1 - (overview_vector <=> %(prompt_vector)s) >= %(match_threshold)s '
    'ORDER BY overview_vector <=> %(prompt_vector)s LIMIT %(match_cnt)s',
    {'prompt_vector': prompt_vector, 'match_threshold': get_matching_threshold(), 'match_cnt': get_matching_count()}
    )

result = cursor.fetchall()

for row in result:
    print(row)

PostgreSQL will pre-filter data using the following columns:

* The `vote_average` column stores a rank from `1` through `10`. 
* The `genres` column is an array of JSON objects stored in the JSONB format. A movie can be categorized by several genres, with a sample value looking as follows - `[{'id': 12, 'name': 'Adventure'}, {'id': 18, 'name': 'Drama'}, {'id': 878, 'name': 'Science Fiction'}])`

Run the `EXPLAIN` statement if you'd like to see the actual execution plan:

In [ ]:
user_prompt = 'A movie about a space adventure.'

prompt_vector = get_embedding(user_prompt)

cursor.execute(
    'EXPLAIN (costs off) SELECT title, vote_average, genres '
    'FROM movie WHERE vote_average >= 7 '
    'AND genres @> \'[{"name": "Science Fiction"}]\' '
    'AND 1 - (overview_vector <=> %(prompt_vector)s) >= %(match_threshold)s '
    'ORDER BY overview_vector <=> %(prompt_vector)s LIMIT %(match_cnt)s',
    {'prompt_vector': prompt_vector, 'match_threshold': get_matching_threshold(), 'match_cnt': get_matching_count()}
    )

result = cursor.fetchall()

for row in result:
    print(row)

The generated plan should look as follows:
```sql
('Limit',)
('  ->  Sort',)
("        Sort Key: ((overview_vector <=> '[0.015902195,-0.03861236,-0.02809557,...]'::vector))",)
('        ->  Seq Scan on movie',)
('              Filter: ((vote_average >= \'7\'::numeric) AND (genres @> \'[{"name": "Science Fiction"}]\'::jsonb) AND ((\'1\'::double precision - (overview_vector <=> \'[0.015902195,-0.03861236,-0.02809557,...]\'::vector)) >= \'0.7\'::double precision))',)

```

The plan indicates that the data is initially filtered by the `vote_average` and `genres` columns, followed by the similarity search on the `overview_vector` column. Note that PostgreSQL may use a different plan if you create an index for any of the columns or if other conditions change.

## Optimize Vector Search With HNSW Index

Currently, the embeddings stored in the `overview_vector` column are not indexed. This means that the database performs an exact nearest neighbor search by comparing a user prompt's vector to all the movies' overview embeddings. You can confirm this by checking the execution plan, which will show the `Seq Scan` (full table scan) access method on the `movie` table:

In [ ]:
user_prompt = 'A movie about a space adventure.'

prompt_vector = get_embedding(user_prompt)

cursor.execute(
    'EXPLAIN (costs off) SELECT title, overview '
    'FROM movie WHERE 1 - (overview_vector <=> %(prompt_vector)s) >= %(match_threshold)s '
    'ORDER BY overview_vector <=> %(prompt_vector)s LIMIT %(match_cnt)s',
    {'prompt_vector': prompt_vector, 'match_threshold': get_matching_threshold(), 'match_cnt': get_matching_count()}
    )

result = cursor.fetchall()

for row in result:
    print(row)

Postgres pgvector supports the IVFFlat and HNSW indexes, which are two of the most widespread index types across vector databases.

Let's create an HNSW index on the embeddings stored in the `overview_vector` column:

In [ ]:
print('Building the index. It might take a minute...')

# Build the index
cursor.execute(
    'CREATE INDEX movie_overview_hnsw_idx ON movie '
    'USING hnsw (overview_vector vector_cosine_ops) '
    'WITH (m = 4, ef_construction = 10)')
conn.commit()

# Update the statistics for the query planner
# to ensure that the index is used for the vector similarity search
conn.autocommit = True
cursor.execute(
    'VACUUM ANALYZE movie')
conn.autocommit = False

print('HNSW Index created.')

Check the execution plan once more for the previous query to ensure that Postgres now expedites the similarity search with the newly created index. You will see the `Index Scan` access method on the `movie` table:

In [ ]:
user_prompt = 'A movie about a space adventure.'

prompt_vector = get_embedding(user_prompt)

cursor.execute(
    'EXPLAIN (costs off) SELECT title, overview '
    'FROM movie WHERE 1 - (overview_vector <=> %(prompt_vector)s) >= %(match_threshold)s '
    'ORDER BY overview_vector <=> %(prompt_vector)s LIMIT %(match_cnt)s',
    {'prompt_vector': prompt_vector, 'match_threshold': get_matching_threshold(), 'match_cnt': get_matching_count()}
    )

result = cursor.fetchall()

for row in result:
    print(row)

## Learn More

Deepen and broaden your knowledge about Postgres pgvector by checking the following resources by PostgreSQL community members:

* pgvector hands-on video tutorial - [Building scalable generative AI apps with PostgreSQL pgvector](https://www.youtube.com/playlist?list=PLYlSOAEcOZ-V2uEJ_nyk32z70YAj4kR5K).
* HNSW index deep dive - [HNSW Indexes with Postgres and pgvector](https://www.crunchydata.com/blog/hnsw-indexes-with-postgres-and-pgvector)  
* pgvector performance tips - [Performance Tips Using Postgres and pgvector](https://www.crunchydata.com/blog/pgvector-performance-for-developers)